In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [5]:
vectorstore = Chroma(persist_directory="./database", embedding_function=OpenAIEmbeddings(model="text-embedding-ada-002"))

In [6]:
len(vectorstore.get()['documents'])

23

In [7]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":3, "lambda_mult":0.7})

In [8]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [10]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  seed=365,
                  max_tokens = 250)

In [17]:
question = "What software do data scientist use?"

In [26]:
chain = {'context':retriever,
         'question':RunnablePassthrough()} | prompt_template | chat | StrOutputParser()
respone = chain.invoke(question)

In [28]:
respone

'Data scientists use a variety of programming languages and software for their work. R and Python are the most popular tools because they can manipulate data and are integrated within multiple data and data science software platforms. They can solve a wide variety of business and data-related problems. Another important software is Hadoop, which is a software framework designed to handle big data by distributing the computational tasks on multiple computers. Furthermore, software like Power BI, SaS, Qlik, and Tableau are excellent choices for business intelligence visualizations.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'